# CRS Index

By default rioxarray stores CRS information as a scalar coordinate in Xarray. This works well because the important metadata persists through all Xarray operations and when it comes time to save an output, we must retrieve this important metadata.

As of Xarray v2022.09.0 it is possible to create custom Indexes (typically a `PandasIndex` is used behind the scenes that determines how dimensional coordinates are queried and aligned). With the new Xindexes interface, we can attach CRS as persistent metadata to a custom `CRSIndex`. Effectively we enhance the default `PandasIndex` by adding associated metadata (CRS) and new functionality like checking CRS compatibility across objects.

This notebook showcases experimental CRSIndex functionality

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import xarray as xr
import geopandas as gpd
import xvec

xr.set_options(
    display_expand_data=False,
    display_expand_indexes=True,
);


In [3]:
# DataArray
DA = xr.open_dataarray("../../test/test_data/input/cog.tif", engine="rasterio")
DA

<xarray.DataArray 'band_data' (band: 1, y: 500, x: 500)>
[250000 values with dtype=float32]
Coordinates:
  * band         (band) int64 1
  * x            (x) float64 1.635e+06 1.635e+06 ... 1.784e+06 1.784e+06
  * y            (y) float64 2.715e+06 2.714e+06 ... 2.565e+06 2.565e+06
    spatial_ref  int64 ...
Attributes:
    AREA_OR_POINT:       Area
    OVR_RESAMPLING_ALG:  NEAREST

Note `x` and `y` dimensions are backed by a corresponding separate `PandasIndex` by default. `spatial_ref` is a non-dimensional scalar coordinate and therefore does not appear in the `Indexes`

In [4]:
DA.rio.crs

CRS.from_wkt('PROJCS["Albers",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378140,298.256999999996,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",23],PARAMETER["longitude_of_center",-96],PARAMETER["standard_parallel_1",29.5],PARAMETER["standard_parallel_2",45.5],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]')

In [5]:
DAcrs = xr.open_dataarray("../../test/test_data/input/cog.tif", engine="rasterio",
                       use_crs_index=True)
DAcrs

<xarray.DataArray 'band_data' (band: 1, y: 500, x: 500)>
[250000 values with dtype=float32]
Coordinates:
  * band         (band) int64 1
    spatial_ref  int64 ...
  * x            (x) float64 1.635e+06 1.635e+06 ... 1.784e+06 1.784e+06
  * y            (y) float64 2.715e+06 2.714e+06 ... 2.565e+06 2.565e+06
Indexes:
    x            CRSIndex (crs=PROJCS["Albers",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS  ...)
    y            CRSIndex (crs=PROJCS["Albers",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS  ...)
Attributes:
    AREA_OR_POINT:       Area
    OVR_RESAMPLING_ALG:  NEAREST

In [6]:
# Or alternatively configure index after opening:
DA.rio.write_crs(DA.rio.crs, use_crs_index=True)

<xarray.DataArray 'band_data' (band: 1, y: 500, x: 500)>
[250000 values with dtype=float32]
Coordinates:
  * band         (band) int64 1
    spatial_ref  int64 0
  * x            (x) float64 1.635e+06 1.635e+06 ... 1.784e+06 1.784e+06
  * y            (y) float64 2.715e+06 2.714e+06 ... 2.565e+06 2.565e+06
Indexes:
    x            CRSIndex (crs=PROJCS["Albers",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS  ...)
    y            CRSIndex (crs=PROJCS["Albers",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS  ...)
Attributes:
    AREA_OR_POINT:       Area
    OVR_RESAMPLING_ALG:  NEAREST

In [7]:
type(DAcrs.xindexes['x'].crs)

rasterio.crs.CRS

Now the spatial relationship between `x` and `y` is indicated by their shared `CRSIndex`, which includes an abbreviated representation (often an EPSG code). The CRS is actually a `rasterio.crs.CRS` object with attributes and methods that propoagates through Xarray operations.

In [8]:
DS = xr.open_dataset("../../test/test_data/input/PLANET_SCOPE_3D.nc", engine="rasterio")
DS

<xarray.Dataset>
Dimensions:      (time: 2, x: 10, y: 10)
Coordinates:
  * time         (time) object 2016-12-19 10:27:29.687763 2016-12-29 12:52:42...
  * x            (x) float64 4.663e+05 4.663e+05 ... 4.663e+05 4.663e+05
  * y            (y) float64 8.085e+06 8.085e+06 ... 8.085e+06 8.085e+06
    spatial_ref  int64 ...
Data variables:
    blue         (time, y, x) float64 ...
    green        (time, y, x) float64 ...
Attributes:
    coordinates:  spatial_ref

In [9]:
DS.rio.crs

CRS.from_epsg(32722)

In [10]:
DSutm = xr.open_dataset("../../test/test_data/input/PLANET_SCOPE_3D.nc", engine="rasterio",
                     use_crs_index=True,
)
DSutm

<xarray.Dataset>
Dimensions:      (time: 2, x: 10, y: 10)
Coordinates:
  * time         (time) object 2016-12-19 10:27:29.687763 2016-12-29 12:52:42...
    spatial_ref  int64 ...
  * x            (x) float64 4.663e+05 4.663e+05 ... 4.663e+05 4.663e+05
  * y            (y) float64 8.085e+06 8.085e+06 ... 8.085e+06 8.085e+06
Data variables:
    blue         (time, y, x) float64 ...
    green        (time, y, x) float64 ...
Indexes:
    x            CRSIndex (crs=EPSG:32722)
    y            CRSIndex (crs=EPSG:32722)
Attributes:
    coordinates:  spatial_ref

In [11]:
# CRS information is retrieved via the `.rio` accessor (In this case UTM https://epsg.io/32722)
DSutm.rio.crs

CRS.from_epsg(32722)

In [12]:
# Or Convert from existing PandasIndex to CRSIndex
DSutm = DS.rio.write_crs(DS.rio.crs, use_crs_index=True)
DSutm

<xarray.Dataset>
Dimensions:      (time: 2, x: 10, y: 10)
Coordinates:
  * time         (time) object 2016-12-19 10:27:29.687763 2016-12-29 12:52:42...
    spatial_ref  int64 0
  * x            (x) float64 4.663e+05 4.663e+05 ... 4.663e+05 4.663e+05
  * y            (y) float64 8.085e+06 8.085e+06 ... 8.085e+06 8.085e+06
Data variables:
    blue         (time, y, x) float64 ...
    green        (time, y, x) float64 ...
Indexes:
    x            CRSIndex (crs=EPSG:32722)
    y            CRSIndex (crs=EPSG:32722)
Attributes:
    coordinates:  spatial_ref

In [13]:
# Access CRS info the same way
DSutm.rio.crs

CRS.from_epsg(32722)

In [14]:
# Or access via xindexes
DSutm.xindexes['x'].crs

CRS.from_epsg(32722)

Now the spatial relationship between `x` and `y` is indicated by their shared `CRSIndex`, which includes an abbreviated representation (often an EPSG code)

## Using CRSIndex

In [15]:
DSll = DS.rio.reproject('EPSG:4326')
DSll

<xarray.Dataset>
Dimensions:      (x: 10, y: 10, time: 2)
Coordinates:
  * x            (x) float64 -51.32 -51.32 -51.32 ... -51.32 -51.32 -51.32
  * y            (y) float64 -17.32 -17.32 -17.32 ... -17.32 -17.32 -17.32
  * time         (time) object 2016-12-19 10:27:29.687763 2016-12-29 12:52:42...
    spatial_ref  int64 0
Data variables:
    blue         (time, y, x) float64 6.611 5.581 0.3996 ... 3.491 5.056 3.368
    green        (time, y, x) float64 7.921 66.15 30.1 ... 21.76 27.29 18.41
Attributes:
    coordinates:  spatial_ref

In [16]:
DS

<xarray.Dataset>
Dimensions:      (time: 2, x: 10, y: 10)
Coordinates:
  * time         (time) object 2016-12-19 10:27:29.687763 2016-12-29 12:52:42...
  * x            (x) float64 4.663e+05 4.663e+05 ... 4.663e+05 4.663e+05
  * y            (y) float64 8.085e+06 8.085e+06 ... 8.085e+06 8.085e+06
    spatial_ref  int64 ...
Data variables:
    blue         (time, y, x) float64 6.611 5.581 0.3996 ... 3.491 5.056 3.368
    green        (time, y, x) float64 7.921 66.15 30.1 ... 21.76 27.29 18.41
Attributes:
    coordinates:  spatial_ref

In [17]:
DS + DSll

<xarray.Dataset>
Dimensions:      (time: 2, x: 0, y: 0)
Coordinates:
  * time         (time) object 2016-12-19 10:27:29.687763 2016-12-29 12:52:42...
  * x            (x) float64 
  * y            (y) float64 
    spatial_ref  int64 0
Data variables:
    blue         (time, y, x) float64 
    green        (time, y, x) float64

With default PandasIndex Xarray will happily align coordinates assuming they belong to the same CRS. In this case, none of the coordinates match (Lat/Lon versus UTM)

In [18]:
# NOTE: if methods create a new DataArray behind the scenes (like rio.reproject) have to chain with write_crs():
DSll = DSutm.rio.reproject('EPSG:4326').rio.write_crs('EPSG:4326', use_crs_index=True)
DSll

<xarray.Dataset>
Dimensions:      (x: 10, y: 10, time: 2)
Coordinates:
  * time         (time) object 2016-12-19 10:27:29.687763 2016-12-29 12:52:42...
    spatial_ref  int64 0
  * x            (x) float64 -51.32 -51.32 -51.32 ... -51.32 -51.32 -51.32
  * y            (y) float64 -17.32 -17.32 -17.32 ... -17.32 -17.32 -17.32
Data variables:
    blue         (time, y, x) float64 6.611 5.581 0.3996 ... 3.491 5.056 3.368
    green        (time, y, x) float64 7.921 66.15 30.1 ... 21.76 27.29 18.41
Indexes:
    x            CRSIndex (crs=EPSG:4326)
    y            CRSIndex (crs=EPSG:4326)
Attributes:
    coordinates:  spatial_ref

In [19]:
DSutm + DSll

ValueError: CRS mismatch between the CRS of index geometries and the CRS of input geometries.
Index CRS: EPSG:32722
Input CRS: EPSG:4326


Combining CRSIndex with PandasIndex currently raises an error, but we might want to allow this:

In [20]:
DSutm + DS

ValueError: cannot re-index or align objects with conflicting indexes found for the following coordinates: 'y' (2 conflicting indexes), 'x' (2 conflicting indexes)
Conflicting indexes may occur when
- they relate to different sets of coordinate and/or dimension names
- they don't have the same type
- they may be used to reindex data along common dimensions

In [21]:
DSll.blue.sel(x=[-51.3173], method='nearest')

<xarray.DataArray 'blue' (time: 2, y: 10, x: 1)>
5.078 2.736 5.267 2.344 5.195 5.714 6.407 ... 3.473 2.622 5.093 4.75 3.218 1.888
Coordinates:
  * time         (time) object 2016-12-19 10:27:29.687763 2016-12-29 12:52:42...
    spatial_ref  int64 0
  * x            (x) float64 -51.32
  * y            (y) float64 -17.32 -17.32 -17.32 ... -17.32 -17.32 -17.32
Indexes:
    x            CRSIndex (crs=EPSG:4326)
    y            CRSIndex (crs=EPSG:4326)
Attributes:
    NETCDF_DIM_EXTRA:        {time}
    NETCDF_DIM_time_DEF:     [2. 6.]
    NETCDF_DIM_time_VALUES:  [     0.       872712.659688]
    units:                   ('DN', 'DN')

In [22]:
# We can ensure we sample or interpolate with points in the same CRS
points = xr.DataArray(coords=dict(x=[-51.31734, -51.31745], y=[-17.3228, -17.3229]), 
                      dims=["x","y"]).rio.write_crs('EPSG:4326', use_crs_index=True)
points

<xarray.DataArray (x: 2, y: 2)>
nan nan nan nan
Coordinates:
    spatial_ref  int64 0
  * x            (x) float64 -51.32 -51.32
  * y            (y) float64 -17.32 -17.32
Indexes:
    x            CRSIndex (crs=EPSG:4326)
    y            CRSIndex (crs=EPSG:4326)

In [23]:
points.x # no CRSIndex here b/c no 'y'...

<xarray.DataArray 'x' (x: 2)>
-51.32 -51.32
Coordinates:
    spatial_ref  int64 0
    x            (x) float64 -51.32 -51.32

In [24]:
# So no CRS check with selection currently:
DSll.sel(x=points.x, method='nearest')

<xarray.Dataset>
Dimensions:      (time: 2, y: 10, x: 2)
Coordinates:
  * time         (time) object 2016-12-19 10:27:29.687763 2016-12-29 12:52:42...
    spatial_ref  int64 0
  * x            (x) float64 -51.32 -51.32
  * y            (y) float64 -17.32 -17.32 -17.32 ... -17.32 -17.32 -17.32
Data variables:
    blue         (time, y, x) float64 5.48 6.611 3.478 ... 1.781 0.1844 1.844
    green        (time, y, x) float64 57.79 7.921 68.12 ... 16.77 53.28 44.67
Indexes:
    x            CRSIndex (crs=EPSG:4326)
    y            CRSIndex (crs=EPSG:4326)
Attributes:
    coordinates:  spatial_ref

## Integrations
If Python libraries store CRS information in predictable / standardized ways we can do neat things combining libraries. This will require some coordination...

In [25]:
gf = gpd.GeoDataFrame(dict(station=['siteA', 'siteB']), 
                      geometry=gpd.points_from_xy(x=[-51.31734, -51.31745], y=[-17.3228, -17.3229]), 
                      crs='EPSG:4326')
gf

,station,geometry
0,siteA,POINT (-51.31734 -17.32280)
1,siteB,POINT (-51.31745 -17.32290)


In [26]:
# Turn points into an Xarray GeometryIndex with xvec (https://xvec.readthedocs.io)
da = xr.DataArray(gf.station, coords=[gf.geometry], dims=["station"])
da = da.xvec.set_geom_indexes("station", crs=gf.crs)
da

<xarray.DataArray 'station' (station: 2)>
'siteA' 'siteB'
Coordinates:
  * station  (station) object POINT (-51.31734 -17.3228) POINT (-51.31745 -17...
Indexes:
    station  GeometryIndex (crs=EPSG:4326)

In [27]:
DS

<xarray.Dataset>
Dimensions:      (time: 2, x: 10, y: 10)
Coordinates:
  * time         (time) object 2016-12-19 10:27:29.687763 2016-12-29 12:52:42...
  * x            (x) float64 4.663e+05 4.663e+05 ... 4.663e+05 4.663e+05
  * y            (y) float64 8.085e+06 8.085e+06 ... 8.085e+06 8.085e+06
    spatial_ref  int64 ...
Data variables:
    blue         (time, y, x) float64 6.611 5.581 0.3996 ... 3.491 5.056 3.368
    green        (time, y, x) float64 7.921 66.15 30.1 ... 21.76 27.29 18.41
Attributes:
    coordinates:  spatial_ref

In [28]:
DSll.xvec.extract_points(da.station, DSll.rio.x_dim, DSll.rio.y_dim)

<xarray.Dataset>
Dimensions:      (time: 2, geometry: 2)
Coordinates:
  * time         (time) object 2016-12-19 10:27:29.687763 2016-12-29 12:52:42...
    spatial_ref  int64 0
  * geometry     (geometry) object POINT (-51.31734 -17.3228) POINT (-51.3174...
Data variables:
    blue         (time, geometry) float64 5.48 4.536 0.5377 6.746
    green        (time, geometry) float64 57.79 23.8 20.33 6.438
Indexes:
    geometry     GeometryIndex (crs=EPSG:4326)
Attributes:
    coordinates:  spatial_ref

In [29]:
# NOTE: could modify extract_points() to work just with `gf` as input
DSll.xvec.extract_points(gf.geometry, x_coords='x', y_coords='y')

<xarray.Dataset>
Dimensions:      (time: 2, geometry: 2)
Coordinates:
  * time         (time) object 2016-12-19 10:27:29.687763 2016-12-29 12:52:42...
    spatial_ref  int64 0
  * geometry     (geometry) object POINT (-51.31734 -17.3228) POINT (-51.3174...
Data variables:
    blue         (time, geometry) float64 5.48 4.536 0.5377 6.746
    green        (time, geometry) float64 57.79 23.8 20.33 6.438
Indexes:
    geometry     GeometryIndex (crs=EPSG:4326)
Attributes:
    coordinates:  spatial_ref

In [30]:
# NOTE: want this to actually fail with a CRS check
DSutm.xvec.extract_points(gf.geometry, x_coords='x', y_coords='y')

<xarray.Dataset>
Dimensions:      (time: 2, geometry: 2)
Coordinates:
  * time         (time) object 2016-12-19 10:27:29.687763 2016-12-29 12:52:42...
    spatial_ref  int64 0
  * geometry     (geometry) object POINT (-51.31734 -17.3228) POINT (-51.3174...
Data variables:
    blue         (time, geometry) float64 0.8441 0.8441 1.844 1.844
    green        (time, geometry) float64 37.09 37.09 44.67 44.67
Indexes:
    geometry     GeometryIndex (crs=EPSG:4326)
Attributes:
    coordinates:  spatial_ref